In [ ]:
import binascii
import serial
import os
import cv2
from  matplotlib import pyplot as plt

In [ ]:
def run_action(cmd):
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    cnt_err = 0
    while 1:
        test_read = ser.read()
        # print('test_read', test_read)
        cnt_err += 1
        if test_read== b'\xa3' or cnt_err == 50:
            break
    
    if cnt_err == 50:
        print('can not get REQ')
    else:
        print('read REQ finished!')
        ser.write(cmd2data(cmd))
        print('send action ok!')
    ser.close()
    
def crc_calculate(package):
    crc = 0
    for hex_data in package:

        b2 = hex_data.to_bytes(1, byteorder='little')
        crc = binascii.crc_hqx(b2, crc)

    return [(crc >> 8), (crc & 255)]    # 校验位两位

def cmd2data(cmd):
    cnt=0
    cmd_list=[]
    for i in cmd:
        
        cnt+=1
        cmd_list+=[ord(i)]
    cmd_list=[0xff,0xff]+[(cnt+5)>>8,(cnt+5)&255]+[0x01,(cnt+1)&255,0x03]+cmd_list
    cmd_list=cmd_list+crc_calculate(cmd_list)
    return cmd_list

def wait_req():
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    while 1:
        test_read=ser.read()
        if test_read== b'\xa3' :
            print('read REQ finished!') 
            break

In [ ]:
def getpos():
    os.system('fswebcam  --no-banner --save temp.jpg -d /dev/video0 2> /dev/null')
    t = cv2.imread('temp.jpg')
    #t = cv2.imread('down2.png')
    print(t.shape)
    w = 160
    h = 160
    t = t[480-h:480,(320-w):(320+w)]
    cv2.imwrite("cut.jpg", t)
    im = t[:,:,::-1] 

    gray = cv2.cvtColor(t, cv2.COLOR_BGR2GRAY)
    im = gray>100

    j = h-1
    run = 0
    while j>=0:
        count = 0
        b = []
        for i in range(2*w):
            if im[j,i] == False: 
                b.append(i)
                count = count + 1
        if count: run = b[count-1]-b[0]
        if 2<run and run<10 and count != 0:
            break
        j = j - 1
    downy = j
    downx = b[int(count/2)]
    # print(downx, downy)r
        j = 0
    run = 0
    while j<h:
        count = 0
        b = []
        for i in range(2*w):
            if im[j,i] == False: 
                b.append(i)
                count = count + 1
        if count: run = b[count-1]-b[0]
        if 2<run and run<10 and count != 0:
            break
        j = j + 1
    upy = j
    upx = b[int(count/2)]
    # print(upx, upy)
    slope = (downx - upx)/(downy - upy)
    b = downx-150
    print(b, slope)
    return (b, slope)
b_thres = 10
s_thres = 0.05

In [ ]:
os.system('sh ./stop_sys_ttyPS0.sh')

In [ ]:
flag1 = False
flag2 = False

while not (flag1 and flag2):
    (b, s) = getpos()
    flag1 = False
    flag2 = False
    
    if b < -b_thres:
        wait_req()
        run_action('turn003L')
    elif b > b_thres:
        wait_req()
        run_action('turn003R')
    else:
        flag1 = True
        
    if s < -s_thres:
        wait_req()
        run_action('Left02move')
    elif s > s_thres:
        wait_req()
        run_action('Right02move')
    else:
        flag2 = True

In [ ]:
wait_req()
run_action('turn001L')

In [ ]:
run_action('Forwalk05')

In [ ]:
def f1L():
    run_action('turn005L')
    wait_req()
    run_action('turn005L')
    wait_req()
    run_action('turn005L')
    wait_req()
    run_action('turn001L')
    wait_req()
    run_action('turn001L')
    wait_req()
    run_action('turn001L')
    wait_req()
    run_action('Stand')